### Scraping Candidates' Sites

In this notebook we'll scrape candidates' websites and store the information for later analysis. We'll store the data locally in a database, which we'll build via Python to practice that skill. Let's get started!

In [1]:
import sqlite3
import requests

from bs4 import BeautifulSoup 
from bs4.element import Comment

import random 
# this package is good for randomization, which we'll 
# use when we're pulling a fraction of the pages. 

import datetime # this is good for working with dates and times. It's a bit confusing though. 

First, let's build a DB for this. Right now I'm assuming we'll have these fields in our only table:

* `dt`: the date and time when we pulled the page.
* `base_url`: the main URL we're pulling from. E.g., www.joebiden.com, for Biden's site. 
* `url`: the specific URL we're pulling from. E.g., https://joebiden.com/joes-story/.
* `text`: the text of the `url`. 
* `pulled`: A boolean that is TRUE if we've tried to pull the text from the URL. This is useful for keeping track. 

In [4]:
db = sqlite3.connect("candidate_websites.db") # feel free to change this to something you like. 
cur = db.cursor()

Now let's create the table in the DB. 

In [ ]:
cur.execute('''DROP TABLE IF EXISTS site_text''')
cur.execute('''CREATE TABLE site_text (
    dt DATETIME, 
    base_url TEXT, 
    url TEXT,
    text TEXT,
    pulled BOOLEAN)''')
db.commit()

Now we'll build off the previous notebook (`Intro to Scraping.ipynb`) to scrape candidates' sites. Let's begin by reading the list of websites.  

In [ ]:
sites = []
with open("candidates_websites.txt",'r') as infile :
    for line in infile :
        sites.append(line.strip())

In [5]:
for candi_site in sites :
    
    r = requests.get(candi_site)
    soup = BeautifulSoup(r.text, 'html.parser')
    all_a_tags = soup.find_all('a')
    
    links = [candi_site] # make sure the main page is in there. 
    
    for link in soup.find_all('a'):
        links.append(link.get('href'))

    # Now we have all the links. Let's just load them into the DB. We'll do the 
    # page pulls and parsing separately. 
    
    for link in set(links) : # wrap a `set` around it so that we don't get duplicates
        new_row = [datetime.datetime.now(),
                   candi_site,
                   link,
                   "", # empty string for text
                   False]
        
        cur.execute('''INSERT INTO site_text (dt,base_url,url,text,pulled) 
               VALUES (?,?,?,?,?)''',new_row)
        
db.commit()

Let's look and see how many pages we have per candidate.

In [6]:
results = cur.execute('''SELECT base_url, count(*) as cnt FROM site_text GROUP BY base_url ORDER BY cnt DESC''')

for row in results :
    print("Original site {} has {} links within it.".format(row[0],row[1]))

Original site https://amyklobuchar.com has 62 links within it.
Original site https://peteforamerica.com/ has 54 links within it.
Original site https://kamalaharris.org/ has 42 links within it.
Original site https://berniesanders.com/ has 40 links within it.
Original site https://betoorourke.com/ has 38 links within it.
Original site https://elizabethwarren.com has 37 links within it.
Original site https://www.donaldjtrump.com/ has 36 links within it.
Original site https://www.yang2020.com/ has 34 links within it.
Original site https://corybooker.com/ has 33 links within it.
Original site https://joebiden.com/ has 33 links within it.
Original site https://www.julianforthefuture.com/ has 26 links within it.


Okay, now that we've identified all the pages we're going to be potentially pulling, let's go through our DB and figure out which ones we want to actually get the text for. 

I'll set a flag for the maximum number of pages we'll pull. We'll use a trick where for doing this. If the flag is negative then we'll do every link. Otherwise we'll set it to the max, unless the candidate has too few pages for that. 

In [7]:
# Our function for pulling visible text
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [9]:
max_pages = -1

for candi_site in sites :
    
    # First, let's get a list of all the links. 
    links = []
    
    sql_query = "SELECT url FROM site_text WHERE base_url = '{}' AND pulled = 0".format(candi_site)    
    link_results = cur.execute(sql_query)

    for row in link_results :
        links.append(row[0])
            
    # Now we take a sample of the links if max_pages is non-negative (and the candidate
    # has enough links).
    if max_pages < 1 or len(links) < max_pages :
        pages_to_pull = links
    else :
        pages_to_pull = random.sample(links,max_pages)
        
    for page in pages_to_pull :
        page_text = '' # initialize it to an empty string
        
        # Many of the pages are relative links. For instance, 
        # elizabethwarren.com has a link "/plans". We'd like 
        # to get that page, so we'll glue them together if 
        # the page doesn't have "http" in it.
        
        if page and not "http" in page : # test to make sure page isn't None
            
            if page[0] == r"/" : # trump links came through "/about/"
                page = page[1:]

            page = "".join([candi_site,page])
        
        try :
            r = requests.get(page)
        except :
            pass 

        if r.status_code == 200 :
            soup = BeautifulSoup(r.text, 'html.parser')
            texts = soup.findAll(text=True)
            visible_texts = filter(tag_visible, texts) 
            page_text = " ".join(t.strip() for t in visible_texts)

        # Now let's update our DB row.
        # We should be able to do this in a single UPDATE statement, 
        # but I had to do it with one for each field we're updating. 
        sql_query = ('UPDATE site_text '
                     'SET text = ? '
                     'WHERE url = ? ' )
        
        # Two items in a list matches up with the two "?"
        cur.execute(sql_query,[page_text,page])
        
        sql_query = ('UPDATE site_text '
                     'SET pulled = 1 '
                     'WHERE url = ? ' )
        
        # Two items in a list matches up with the two "?"
        cur.execute(sql_query,[page])

        sql_query = ('UPDATE site_text '
                     'SET dt = ? '
                     'WHERE url = ? ' )
        
        # Two items in a list matches up with the two "?"
        cur.execute(sql_query,[datetime.datetime.now(),page])
        
    print("Completed pull for {}.".format(candi_site))

db.commit()

Completed pull for https://www.donaldjtrump.com/.
